In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
# LangSmith 추적을 설정합니다. 
from langchain_teddynote import logging

logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


In [5]:
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, \
    HumanMessagePromptTemplate, AIMessagePromptTemplate

prompt_template = PromptTemplate.from_template("{country}의 수도는 어디인가요?")
prompt_template

PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [9]:
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [11]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-4o-mini",
    max_tokens=2048,
    temperature=0.1,
)

### LCEL(LangChain Expression Language)

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.

In [13]:
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template("{topic} 에 대해 쉽게 설명해주세요.")

model = ChatOpenAI()

chain = prompt | model | StrOutputParser()

chain.invoke({"topic": "인공지능 모델의 학습 원리"})

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 사용하여 패턴이나 규칙을 학습하는 과정입니다. 먼저 모델은 입력 데이터를 받아들이고, 이 데이터를 분석하여 패턴을 찾습니다. 이후 모델은 이러한 패턴을 바탕으로 예측이나 분류를 수행하게 됩니다. \n\n모델은 학습 데이터를 사용하여 자신을 업데이트하고, 오차를 최소화하도록 조정해나갑니다. 이 과정을 반복하면서 모델은 더 정확하고 효율적으로 작동하도록 개선됩니다. 이러한 과정을 통해 인공지능 모델은 데이터를 기반으로 패턴을 학습하고 문제를 해결할 수 있게 됩니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 237, 'prompt_tokens': 33, 'total_tokens': 270, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-40b5fb5e-f13a-484b-99ea-a47159724e14-0', usage_metadata={'input_tokens': 33, 'output_tokens': 237, 'total_tokens': 270, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### 스트림 출력

In [15]:
from langchain_teddynote.messages import stream_response

answer = chain.stream("인공지능의 학습 원리")

stream_response(answer)

인공지능은 데이터를 입력받아 패턴을 학습하고 이를 기반으로 판단이나 예측을 수행하는 시스템입니다. 

인공지능의 학습 원리는 크게 지도 학습, 비지도 학습, 강화 학습으로 나뉩니다. 

- 지도 학습: 입력 데이터와 정답 레이블을 함께 제공하여 모델을 학습시키는 방식입니다. 모델은 입력 데이터와 정답 레이블 간의 관계를 학습하고, 새로운 데이터에 대해 정확한 예측을 할 수 있습니다. 예를 들어, 고양이와 개의 이미지를 입력하고 정답 레이블을 제공하여 고양이와 개를 구분하는 모델을 학습시킬 수 있습니다.

- 비지도 학습: 정답 레이블이 제공되지 않는 데이터를 이용하여 모델을 학습시키는 방식입니다. 모델은 데이터의 패턴이나 구조를 스스로 찾아내고, 데이터를 그룹화하거나 차원을 축소하는 등의 작업을 수행할 수 있습니다. 예를 들어, 고객의 구매 기록을 이용하여 비슷한 구매 패턴을 가진 고객을 찾아내는 클러스터링 작업을 수행할 수 있습니다.

- 강화 학습: 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방식입니다. 모델은 특정 행동을 취하고 그 결과로 받는 보상을 최대화하기 위해 학습하며, 시간이 지남에 따라 최적의 행동을 찾아가는 과정을 거칩니다. 예를 들어, 게임을 플레이하는 인공지능이 보상을 최대화하기 위해 최적의 전략을 학습하는 것을 예로 들 수 있습니다.

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("당신은 영어를 가르치는 10년차 영어 선생님입니다. 상황에 따라 [FORMAT]에 영어 회화를 작성해 주세요."),
    HumanMessagePromptTemplate.from_template("상황: {question}"),
    AIMessagePromptTemplate.from_template("""
    FORMAT:
    - 영어 회화:
    - 한글 해석:
    """)
])
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

response = chain.stream({"question" : "공항 입국 수속"})

stream_response(response)

- 영어 회화:
  - Customs Officer: "Welcome to the country! May I see your passport, please?"
  - Traveler: "Sure, here it is."
  - Customs Officer: "What is the purpose of your visit?"
  - Traveler: "I’m here for vacation. I'll be staying for two weeks."
  - Customs Officer: "Great! Do you have any items to declare?"
  - Traveler: "No, I don’t have anything to declare."
  - Customs Officer: "Alright, enjoy your stay!"

- 한글 해석:
  - 세관 직원: "나라에 오신 것을 환영합니다! 여권 좀 보여주시겠어요?"
  - 여행자: "네, 여기 있습니다."
  - 세관 직원: "방문 목적이 무엇인가요?"
  - 여행자: "휴가를 위해 왔습니다. 2주 동안 있을 예정입니다."
  - 세관 직원: "좋아요! 신고할 물품이 있으신가요?"
  - 여행자: "아니요, 신고할 물품은 없습니다."
  - 세관 직원: "알겠습니다. 즐거운 여행 되세요!"

### LCEL 인터페이스
- `stream`: 응답의 청크를 스트리밍합니다.
- `invoke`: 입력에 대해 체인을 호출합니다.
- `batch`: 입력 목록에 대해 체인을 호출합니다.
- `astream`: 비동기적으로 응답의 청크를 스트리밍합니다.
- `ainvoke`: 비동기적으로 입력에 대해 체인을 호출합니다.
- `abatch`: 비동기적으로 입력 목록에 대해 체인을 호출합니다.
- `astream_log`: 최종 응답뿐만 아니라 발생하는 중간 단계를 스트리밍합니다.

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI(model="gpt-4o-mini")
prompt = PromptTemplate.from_template("{topic} 에 대하여 3문장으로 설명해줘.")
chain = prompt | model | StrOutputParser()

### stream : 실시간 출력
- 이 함수는 chain.stream 메서드를 사용하여 주어진 토픽에 대한 데이터 스트림을 생성하고, 이 스트림을 반복하여 각 데이터의 내용(content)을 즉시 출력합니다. end="" 인자는 출력 후 줄바꿈을 하지 않도록 설정하며, flush=True 인자는 출력 버퍼를 즉시 비우도록 합니다.

In [26]:
for token in chain.stream({"topic": "멀티모달"}):
    print(token, end="", flush=True)
    
stream_response(chain.stream({"topic": "멀티모달"}))

멀티모달은 다양한 종류의 데이터를 통합하여 처리하는 기술을 의미합니다. 예를 들어, 텍스트, 이미지, 오디오 등 여러 형태의 정보를 동시에 활용하여 더 풍부하고 정확한 분석이나 결과를 도출할 수 있습니다. 이러한 접근은 인공지능, 특히 딥러닝 분야에서 자연어 처리(NLP)와 컴퓨터 비전(CV) 등의 연구에 많이 활용됩니다.멀티모달은 다양한 유형의 데이터를 동시에 처리하고 분석하는 접근 방식을 의미합니다. 예를 들어, 텍스트, 이미지, 음성 등의 서로 다른 모드를 결합하여 정보를 이해하고 활용할 수 있습니다. 이러한 방법은 인공지능 및 머신러닝 분야에서 더욱 중요해지고 있으며, 보다 풍부하고 정확한 결과를 도출하는 데 기여하고 있습니다.

### invoke: 호출
- chain 객체의 invoke 메서드는 주제를 인자로 받아 해당 주제에 대한 처리를 수행합니다.

In [27]:
chain.invoke({"topic": "ChatGPT"})

'ChatGPT는 OpenAI가 개발한 대화형 인공지능 모델로, 자연어 처리 기술을 기반으로 사용자의 질문이나 요청에 대해 인간처럼 대화할 수 있습니다. 이 모델은 방대한 양의 텍스트 데이터를 학습하여 다양한 주제에 대해 정보를 제공하고, 창의적인 글쓰기, 문제 해결 등 여러 가지 작업을 수행할 수 있습니다. 또한, 지속적인 개선과 업데이트를 통해 더욱 정확하고 유용한 대화를 제공하는 것을 목표로 하고 있습니다.'

In [ ]:
### batch: 배치(단위 실행)
함수 chain.batch는 여러 개의 딕셔너리를 포함하는 리스트를 인자로 받아, 각 딕셔너리에 있는 topic 키의 값을 사용하여 일괄 처리를 수행합니다.

In [ ]:
chain.batch([{"topic": "ChatGPT"}, {"topic": "Instagram"}])

### async stream: 비동기 스트림
- 함수 chain.astream은 비동기 스트림을 생성하며, 주어진 토픽에 대한 메시지를 비동기적으로 처리합니다.

In [28]:
async for token in chain.astream({"topic": "YouTube"}):
    print(token, end="", flush=True)

YouTube는 사용자가 동영상을 업로드, 공유 및 시청할 수 있는 세계 최대의 비디오 플랫폼입니다. 2005년에 설립된 이 플랫폼은 개인 사용자뿐만 아니라 기업과 콘텐츠 제작자에게도 널리 사용되며, 다양한 주제의 콘텐츠가 제공됩니다. 사용자는 댓글, 좋아요, 구독 기능을 통해 서로 소통하고, 알고리즘에 기반한 추천 시스템을 통해 새로운 동영상을 발견할 수 있습니다.

### async invoke: 비동기 호출

In [29]:
my_process = chain.ainvoke({"topic": "NVDA"})

In [30]:
await my_process

'NVIDIA Corporation (NVDA)은 그래픽 처리 장치(GPU) 및 인공지능(AI) 기술을 선도하는 글로벌 기업입니다. 이 회사는 게임, 데이터 센터, 자율주행차, 헬스케어 등 다양한 분야에 걸쳐 혁신적인 솔루션을 제공하며, AI와 머신러닝 분야에서의 성장이 두드러집니다. 또한, NVIDIA의 GPU는 고성능 컴퓨팅과 시각화에 필수적인 요소로 자리 잡고 있습니다.'

### async batch: 비동기 배치
- 함수 abatch는 비동기적으로 일련의 작업을 일괄 처리합니다.

In [ ]:
my_abatch_process = chain.abatch(
    [{"topic": "YouTube"}, {"topic": "Instagram"}, {"topic": "Facebook"}]
)

await my_abatch_process

## 07. Runnable
### 데이터를 효과적으로 전달하는 방법
- `RunnablePassthrough`: 입력을 변경하지 않거나 추가 키를 더하여 전달할 수 있습니다.
- `RunnablePassthrough()`: 단독으로 호출되면, 단순히 입력을 받아 그대로 전달합니다.
- `RunnablePassthrough.assign(...)`: 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가합니다.

In [33]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


prompt = PromptTemplate.from_template("{num} 의 10배는?")
llm = ChatOpenAI(temperature=0)

chain = prompt | llm

In [34]:
from langchain_core.runnables import RunnablePassthrough

runnable_chain = {"num": RunnablePassthrough()} | prompt | ChatOpenAI()

runnable_chain.invoke(10)

AIMessage(content='10의 10배는 100입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 16, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-79d9aba4-7c76-42a8-892a-ca884ea7714e-0', usage_metadata={'input_tokens': 16, 'output_tokens': 12, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [36]:
# 입력 키: num, 할당(assign) 키: new_num
(RunnablePassthrough.assign(num=lambda x: x["num"] * 3)).invoke({"num": 1})

{'num': 3}